## Test Harmonization

The purpose of this notebook is to assert that harmonization is being performed as expected. Specifically, we will compare the outputs of both neurocombat (Python) and the existing R-based harmonization approach to determine equivalence. We will also try to visualize as much as possible regarding the harmonization, for sanity's sake.

**Requires** the generated `..\data\harmonize_data.csv` from `..\01_Data_Setup.ipynb` with the columns _Connectome_ and _R_Comfam_


In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF
from neuroCombat import neuroCombat
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

data = pd.read_pickle("..\data\harmonize_data.pkl")
data.sample(2)

In [ ]:
print(data["Connectome"][0].max())
print(data["Connectome"][0].min())

In [ ]:
# Obtain neuroCombat harmonization vectors
connectome_vectors = np.transpose(np.vstack(data["R_Comfam"]))
covars = pd.DataFrame(
    {
        "gender": data["Sex"].map({"F": 1, "M": 2, "f": 1, "m": 2}).values,
        "age": data["Age"].values,
        "site": data["Site"].values.astype(np.int8),
    }
)
combat = neuroCombat(
    dat=connectome_vectors, covars=covars, batch_col="site", categorical_cols=["gender"]
)
combat["data"]

In [ ]:
# Element-wise percent difference with a figure to demonstrate the top 20 subject-level differences in connectomes between harmonization methods
comparison_vectors = np.transpose(np.vstack(data["R_Comfam"]))
percent_diff = (
    np.abs(combat["data"] - comparison_vectors)
    / (np.abs(combat["data"]) + np.abs(comparison_vectors))
    * 100
)

means = np.mean(percent_diff, axis=0)
stddevs = np.std(percent_diff, axis=0)

sorted_indices = np.argsort(means)[::-1]

top_20_indices = sorted_indices[:20]
top_20_means = means[top_20_indices]
top_20_stddevs = stddevs[top_20_indices] / 2

overall_mean = np.mean(means)
overall_stddev = np.std(means) / 2


print(top_20_means)

plt.figure(figsize=(10, 6))

bars = plt.barh(
    range(20),
    top_20_means,
    color="skyblue",
    edgecolor="black",
    label="Individual subject",
)
overall_bar = plt.barh(
    20, overall_mean, color="orange", edgecolor="black", label="Average of all subjects"
)
plt.errorbar(
    top_20_means,
    range(20),
    xerr=top_20_stddevs,
    fmt="x",
    color="black",
    markersize=5,
    label="STDEV",
    elinewidth=1,
)
plt.errorbar(
    overall_mean,
    20,
    xerr=overall_stddev,
    fmt="x",
    color="black",
    markersize=5,
    elinewidth=1,
)

plt.yticks(range(21), [f"Subject {i+1}" for i in top_20_indices] + ["AVERAGE"])
plt.xlabel("Mean Percent Difference (%)")
plt.title(
    "Top 20 Greatest Mean Percent Differences between\n Harmonized Connectomes from R_Comfam and neuroCombat"
)
plt.gca().invert_yaxis()
plt.xlim([0, 100])
plt.grid(True, axis="x", linestyle="--", alpha=0.7)

plt.legend(loc="lower right")

plt.show()

In [ ]:
# Inspect the top 10 subset of the top 20 further, and view differences in distributions for the connectome vectors

top_10_indices = sorted_indices[:10]
top_10_means = means[top_10_indices]
top_10_stddevs = stddevs[top_10_indices] / 2

fig, axes = plt.subplots(2, 5, figsize=(20, 10))

all_data = np.concatenate([combat["data"], comparison_vectors], axis=1)
print(all_data.shape)
bin_edges = np.histogram_bin_edges(all_data, bins=100)

for i, idx in enumerate(top_10_indices):
    row = i // 5
    col = i % 5
    ax = axes[row, col]

    subject_data_combat = combat["data"][:, idx]
    subject_data_comparison = comparison_vectors[:, idx]

    ax.hist(
        subject_data_combat,
        bins=bin_edges,
        alpha=0.5,
        color="red",
        label="neuroCombat Connectome",
        edgecolor="black",
    )
    ax.hist(
        subject_data_comparison,
        bins=bin_edges,
        alpha=0.5,
        color="green",
        label="Unharmonized Connectome",
        edgecolor="black",
    )
    ax.set_title(f"Subject {idx+1}")

    ax.set_xlim([-5, 5])
    ax.set_ylim([0, 6000])
    ax.grid(alpha=0.25)

    if row > 0:
        ax.set_xlabel("Edge Value")

    if col == 0:
        ax.set_ylabel("Frequency")

fig.legend(
    ["neuroCombat Connectome", "Unharmonized Connectome"],
    loc="lower right",
    fontsize=10,
    frameon=True,
)
fig.suptitle("Distributions of Top 10 Subjects in Figure 1", fontsize=16)

In [ ]:
# norm comparison
combat_norm = np.linalg.norm(combat["data"], ord="fro")
r_comfam_norm = np.linalg.norm(comparison_vectors, ord="fro")
print(combat_norm, r_comfam_norm)
print(
    "Norm Percent Difference: ",
    np.abs(combat_norm - r_comfam_norm) / (combat_norm + r_comfam_norm) * 100,
    "%",
)

In [ ]:
# Kolmogorov-Smirnov
combat_data_flat = combat["data"].flatten()
comparison_vectors_flat = comparison_vectors.flatten()

neurocombat_cdf = ECDF(combat_data_flat)
rcomfam_cdf = ECDF(comparison_vectors_flat)

k = np.argmax(np.abs(neurocombat_cdf.y - rcomfam_cdf.y))
ks_stat = np.abs(neurocombat_cdf.y[k] - rcomfam_cdf.y[k])

plt.plot(neurocombat_cdf.x, neurocombat_cdf.y, label="neuroCombat")
plt.plot(rcomfam_cdf.x, rcomfam_cdf.y, label="Unharmonized")

y = (neurocombat_cdf.y[k] + rcomfam_cdf.y[k]) / 2
# plt.errorbar(x=neurocombat_cdf.x[k], y=y, yerr=k/2, color='k',
#              capsize=5, mew=3, label=f"Test statistic: {ks_stat:.4f}")

plt.legend(loc="upper left")
plt.grid(alpha=0.75)
plt.xlabel("Connectome Value")
plt.ylabel("Density")
plt.title("CDF")

In [ ]:
# Batch Detect for Batches
from batchdetect.batchdetect import BatchDetect

bd = BatchDetect(covars, np.transpose(connectome_vectors))
bd.low_dim_visualization("pca")

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns

adata = bd.adata
# sc.tl.pca(adata)

pca_scores = adata.obsm["X_pca"]
pca_df = pd.DataFrame(pca_scores)

outlier_threshold = 72
outlier_rows = pca_df[pca_df.iloc[:, 1].abs() > outlier_threshold]
outlier_data = adata.X[outlier_rows.index]
if isinstance(outlier_data, np.ndarray):
    outlier_data_df = pd.DataFrame(outlier_data)
else:
    outlier_data_df = pd.DataFrame(outlier_data.toarray())
print("Outlier data for subjects with PC2 values exceeding the threshold:")
print(outlier_data_df.head())

# n_components = pca_df.shape[1]
# fig, axes = plt.subplots(n_components, 1, figsize=(8, 2 * n_components))
# if n_components == 1:
#     axes = [axes]

# for i in range(n_components):
#     ax = axes[i]
#     sns.histplot(
#         pca_df.iloc[:, i], bins=30, kde=True, color="skyblue", edgecolor="black", ax=ax
#     )
#     ax.axvline(
#         x=outlier_threshold,
#         color="red",
#         linestyle="--",
#         label=f"Outlier Threshold ({outlier_threshold})",
#     )
#     ax.set_xlabel(f"PC{i+1} Values")
#     ax.set_ylabel("Frequency")
#     ax.set_title(f"Histogram of PC{i+1} Values")
#     ax.legend()

# plt.tight_layout()
# plt.show()

In [ ]:
count = 0
for row in data.itertuples(index=False):
    print(count, row.R_Comfam, row.Subject_ID)
    count = count + 1

In [ ]:
bd.low_dim_visualization("tsne")

In [ ]:
import numpy as np
import pandas as pd

# Assuming you already ran:
# bd.low_dim_visualization("tsne")

# Extract tSNE coordinates
tsne_scores = adata.obsm["X_tsne"]
tsne_df = pd.DataFrame(tsne_scores, columns=["tSNE1", "tSNE2"])

# Define your outlier threshold for tSNE2 (example value)
outlier_threshold = 400  # adjust based on your data spread

# Find outlier rows (subjects) where abs(tSNE2) exceeds the threshold
outlier_rows = tsne_df[tsne_df["tSNE2"].abs() > outlier_threshold]

print(
    f"Found {len(outlier_rows)} outliers based on tSNE2 threshold = {outlier_threshold}"
)
print(outlier_rows)

# Get feature data for those subjects
outlier_data = adata.X[outlier_rows.index]

# Convert to DataFrame (in case it's sparse)
if isinstance(outlier_data, np.ndarray):
    outlier_data_df = pd.DataFrame(outlier_data)
else:
    outlier_data_df = pd.DataFrame(outlier_data.toarray())

# Optionally print or save outlier feature data
print("\nFeature vectors for outlying subjects (tSNE2):")
print(outlier_data_df.head())

In [ ]:
max(tsne_df["tSNE2"])

In [ ]:
bd.low_dim_visualization("umap")

In [ ]:
bd.prince_plot(5)